In [2]:
import pickle
import pandas as pd
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/RenataOrzc/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as RenataOrzc

Initialized MLflow to track repo "RenataOrzc/nyc-taxi-time-prediction"

Repository RenataOrzc/nyc-taxi-time-prediction initialized!

https://dagshub.com/RenataOrzc/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/5237519f0cf74c8e961551526c9bca87', creation_time=1726875667929, experiment_id='0', last_update_time=1726875667929, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

Importar las librerias necesarias

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np

In [12]:
mlflow.sklearn.autolog() 

def objective(params):
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family","random_forest")
        mlflow.log_params(params)

        rf_model = RandomForestRegressor(
            n_estimators = int(params['n_estimators']),
            max_depth = int(params['max_depth']),
            min_samples_split = int(params['min_samples_split']),
            min_samples_leaf = int(params['min_samples_leaf']),
            random_state = 20
        )

        rf_model.fit(X_train,y_train)

        y_pred = rf_model.predict(X_val)

        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        mlflow.log_metric("rmse",rmse)

    return {'loss':rmse, 'status': STATUS_OK}

In [15]:
search_space = {
    'n_estimators': hp.quniform('n_estimators',100,150,1),
    'max_depth': hp.quniform('max_depth',10,15,1),
    'min_samples_split': hp.quniform('min_samples_split',4,10,1),
    'min_samples_leaf': hp.quniform('min_samples_leaf',1,3,1)
}

In [16]:
with mlflow.start_run(run_name='Random Forest Tata', nested=True):
    best_params= fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
        
    )

    mlflow.log_params(best_params)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/23 13:38:09 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/23 13:38:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/2x/ywqv7yz10h9cq_ql_b4l1vpw0000gn/T/tmp5roxbnee/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 

2024/09/23 13:38:25 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID c8d93606f3eb4c9fa16880915991dda0. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/23 13:38:25 INFO mlflow.tracking._tracking_

 10%|█         | 1/10 [00:17<02:40, 17.81s/trial, best loss: 5.4014531039870155]

2024/09/23 13:38:26 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/23 13:38:37 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/2x/ywqv7yz10h9cq_ql_b4l1vpw0000gn/T/tmpo1azszs8/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 

2024/09/23 13:38:40 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID c0ef2ba8aae64aa6916d51289d5e33fc. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/23 13:38:41 INFO mlflow.tracking._tracking_

 20%|██        | 2/10 [00:33<02:10, 16.34s/trial, best loss: 5.4014531039870155]

...

2024/09/23 13:40:17 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/23 13:40:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/2x/ywqv7yz10h9cq_ql_b4l1vpw0000gn/T/tmpro97zjno/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 

2024/09/23 13:40:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 286ec19dc1394eacbb7f7d18be71681b. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/23 13:40:35 INFO mlflow.tracking._tracking_

 90%|█████████ | 9/10 [02:27<00:16, 16.79s/trial, best loss: 5.384427961181591]

2024/09/23 13:40:36 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/09/23 13:40:49 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/2x/ywqv7yz10h9cq_ql_b4l1vpw0000gn/T/tmp6q0lw764/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 

2024/09/23 13:40:52 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID c3fed7697eed4feab58e0c0ae93a2a50. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

2024/09/23 13:40:52 INFO mlflow.tracking._tracking_

100%|██████████| 10/10 [02:44<00:00, 16.49s/trial, best loss: 5.384427961181591]


2024/09/23 13:40:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Tata at: https://dagshub.com/RenataOrzc/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/40fc056fe94d4d3c8bcd752050cb4a99.
2024/09/23 13:40:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/RenataOrzc/nyc-taxi-time-prediction.mlflow/#/experiments/0.


In [17]:
run_id = '40fc056fe94d4d3c8bcd752050cb4a99'
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri= run_uri,
    name='nyc-taxi-model'
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/09/23 13:43:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 2
Created version '2' of model 'nyc-taxi-model'.
